In [1]:
from typing import Optional

import ase
from ase.calculators.calculator import Calculator
from sella import Constraints, Internals, Sella


def optimize(
    atms_obj: ase.Atoms,
    calc: Calculator,
    order: int = 0,
    ints_obj: Optional[Internals] = None,
    in_place: bool = False,
):
    """Optimize a geometry using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param order: 0 = minimum | 1 = saddle point
    :param ints_obj: A Sella Internals object, possibly involving constraints
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    atms_obj.calc = calc

    # Initialize and run the optimization
    dyn = Sella(
        atms_obj, order=order, internal=(True if ints_obj is None else ints_obj)
    )
    dyn.run()

    return atms_obj


def optimize_minimum(atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False):
    """Optimize a minimum-energy structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return optimize(atms_obj, calc, order=0, in_place=in_place)


def optimize_ts(atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False):
    """Optimize a TS/saddle-point structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return optimize(atms_obj, calc, order=1, in_place=in_place)


def optimize_constrained(
    atms_obj: ase.Atoms,
    calc: Calculator,
    const_coos: Optional[list[tuple[int, ...]]] = None,
    in_place: bool = False,
):
    """Optimize a structure subject to internal coordinate constraints using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param const_coos: Optionally, constrain a set of coordinates
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    # Set up constraints
    const_obj = Constraints(atms_obj)
    for const_coo in const_coos:
        print("Applying constraint:", const_coo)
        if len(const_coo) == 2:
            const_obj.fix_bond(const_coo)
        elif len(const_coo) == 3:
            const_obj.fix_angle(const_coo)
        elif len(const_coo) == 4:
            const_obj.fix_dihedral(const_coo)

    # Set up internal coordinates
    ints_obj = Internals(atms_obj, cons=const_obj)
    ints_obj.find_all_bonds()
    ints_obj.find_all_angles()
    ints_obj.find_all_dihedrals()

    # Note: For whatever reason, the Atoms object must be identical to the one passed in
    # to Constraints and Internals, so we *must* set in_place=True here
    return optimize(atms_obj, calc, ints_obj=ints_obj, in_place=True)

In [2]:
import automol


def log_geometry_info(geo, coo, const_coos, gra=None):
    """Print and display information about a geometry

    :param geo: An automol geometry data structure
    :param coo: The reaction/scan coordinate
    :param const_coos: Any constrained coordinates
    """
    # Display the geometry
    automol.geom.display(geo, gra=gra)

    # Print the distance for the reaction/scan coordinate
    dist = automol.geom.distance(geo, *coo, angstrom=True)
    print(f" - {coo} distance: {dist}")

    # Print distances for the constraint coordinates
    for const_coo in const_coos:
        const_dist = automol.geom.distance(geo, *const_coo, angstrom=True)
        print(f" - constrained {const_coo} distance: {const_dist}")

In [3]:
from tblite.ase import TBLite

rsmi, psmi = ("CCCO[O]", "[CH2]CCOO")
rsmis = automol.smiles.split(rsmi)
psmis = automol.smiles.split(psmi)

# Get the TS guess structure
rxn, *_ = automol.reac.from_smiles(rsmis, psmis, stereo=False, struc_typ="geom")
ts_gra = automol.reac.ts_graph(rxn)
geo0 = automol.reac.ts_structure(rxn)
coo = automol.reac.scan_coordinate(rxn)
const_coos = automol.reac.constraint_coordinates(rxn)
all_const_coos = (coo,) + const_coos

# Optimize using Sella
atms_obj0 = automol.geom.ase_atoms(geo0)
# atms_obj = optimize_minimum(atms_obj=atms_obj0, calc=TBLite(method="GFN1-xTB"))
atms_obj = optimize_ts(atms_obj=atms_obj0, calc=TBLite(method="GFN1-xTB"))
# atms_obj = optimize_constrained(
#     atms_obj=atms_obj0, calc=TBLite(method="GFN1-xTB"), const_coos=all_const_coos
# )
geo = automol.geom.from_ase_atoms(atms_obj)

/home/avcopan/code/amech-dev/src/autochem/sella-repo/notebook-examples/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -18.99057833730  -1.9167150E+01   5.3793479E-01
      2     -19.21567441147  -2.2509607E-01   3.3989348E-01
      3     -19.20704254664   8.6318648E-03   1.6074957E-01
      4     -19.22777691335  -2.0734367E-02   5.6557343E-02
      5     -19.23666016733  -8.8832540E-03   2.6427568E-02
      6     -19.23930951642  -2.6493491E-03   4.2408560E-03
      7     -19.23929384216   1.5674262E-05   2.6364911E-03
      8     -19.23929202588   1.8162756E-06   2.8004935E-03
      9     -19.23932836414  -3.6338255E-05   3.8978647E-04
     10     -19.23932889673  -5.3258707E-07   2.9796322E-04
     11     -19.23932929923  -4.0250320E-07   1.0188038E-04
     12     -19.23932935164  -5.2415668E-08   1.4591912E-05
------------------------------------------------------------

 total:                             

In [4]:
# Visualize the results
automol.graph.display(ts_gra, label=True, exp=True)

print("\nStarting geometry:")
log_geometry_info(geo0, coo, const_coos, gra=ts_gra)

print("\nFinal geometry:")
log_geometry_info(geo, coo, const_coos, gra=ts_gra)


Starting geometry:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

 - (4, 5) distance: 1.824046106909827
 - constrained (3, 4) distance: 1.426068438639302

Final geometry:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

 - (4, 5) distance: 1.3398861343274897
 - constrained (3, 4) distance: 1.365507606132882
